In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from utils import *

In [2]:
from django.db.models import Q, F, Count, Min, Max, Sum, Avg
from django.db.models.functions import Round
from django.db import connection
from customer_db.models import Occupation, Person

In [3]:
o = orm_to_df(Occupation.objects.all().values())
p = orm_to_df(Person.objects.all().values())
print(o.shape, p.shape)

(2, 2) (2, 3)


In [4]:
sql_to_df(sql_raw("SELECT * from person;"))

,person_id,name,occupation_title
0,1000,Person1,None
1,1001,Person2,Doctor


In [5]:
sql_to_df(sql_raw("SELECT * from occupation;"))

,title,avg_salary
0,School Teacher,40000
1,Doctor,80000


# INNER JOIN 
## lhs = Person, rhs = Occupation

In [6]:
q = """
Select * FROM person p
INNER JOIN occupation o
on o.title == p.occupation_title
"""
sqlq =  sql_raw(q) 
ji = sql_to_df(sqlq)
print(ji.shape)
ji

(1, 5)


,person_id,name,occupation_title,title,avg_salary
0,1001,Person2,Doctor,Doctor,80000


In [7]:
ormq =  Person.objects.filter(occupation_title__isnull=False).select_related('occupation_title')
print(ormq.query)

SELECT "person"."person_id", "person"."name", "person"."occupation_title", "occupation"."title", "occupation"."avg_salary" FROM "person" INNER JOIN "occupation" ON ("person"."occupation_title" = "occupation"."title") WHERE "person"."occupation_title" IS NOT NULL


In [8]:
ormq =  Person.objects.filter(occupation_title__isnull=False)\
.values("person_id", "name", "occupation_title", avg_salary=F("occupation_title__avg_salary"))
print(ormq.query)

SELECT "person"."person_id", "person"."name", "person"."occupation_title", "occupation"."avg_salary" AS "avg_salary" FROM "person" INNER JOIN "occupation" ON ("person"."occupation_title" = "occupation"."title") WHERE "person"."occupation_title" IS NOT NULL


In [9]:
orm_to_df(ormq)

,person_id,name,occupation_title,avg_salary
0,1001,Person2,Doctor,80000.00


# INNER JOIN (tables reversed)

In [10]:
q = """
Select * FROM occupation o
INNER JOIN person p 
on p.occupation_title == o.title
"""
sqlq =  sql_raw(q) 
ji = sql_to_df(sqlq)
print(ji.shape)
ji

(1, 5)


,title,avg_salary,person_id,name,occupation_title
0,Doctor,80000,1001,Person2,Doctor


In [11]:
ormq =  Occupation.objects\
.values("title", "avg_salary", person_id=F("person__person_id"),
        name=F("person__name"), occupation_title=F("person__occupation_title"),).filter(person_id__isnull=False)
print(ormq.query)

SELECT "occupation"."title", "occupation"."avg_salary", "person"."person_id" AS "person_id", "person"."name" AS "name", "person"."occupation_title" AS "occupation_title" FROM "occupation" LEFT OUTER JOIN "person" ON ("occupation"."title" = "person"."occupation_title") WHERE "person"."person_id" IS NOT NULL


In [12]:
orm_to_df(ormq)

,title,avg_salary,person_id,name,occupation_title
0,Doctor,80000.00,1001,Person2,Doctor


# LEFT OUTER JOIN

In [13]:
q = """
Select * FROM person p
LEFT OUTER JOIN occupation o
on o.title == p.occupation_title
"""
sqlq =  sql_raw(q) 
je = sql_to_df(sqlq)
print(je.shape)
je

(2, 5)


,person_id,name,occupation_title,title,avg_salary
0,1000,Person1,None,None,NaN
1,1001,Person2,Doctor,Doctor,80000.0


In [14]:
ormq =  Person.objects.select_related('occupation_title')
print(ormq.query)

SELECT "person"."person_id", "person"."name", "person"."occupation_title", "occupation"."title", "occupation"."avg_salary" FROM "person" LEFT OUTER JOIN "occupation" ON ("person"."occupation_title" = "occupation"."title")


In [15]:
ormq =  Person.objects\
.values("person_id", "name", "occupation_title", avg_salary=F("occupation_title__avg_salary"))
print(ormq.query)

SELECT "person"."person_id", "person"."name", "person"."occupation_title", "occupation"."avg_salary" AS "avg_salary" FROM "person" LEFT OUTER JOIN "occupation" ON ("person"."occupation_title" = "occupation"."title")


In [16]:
print(orm_to_df(ormq).shape)
orm_to_df(ormq)

(2, 4)


,person_id,name,occupation_title,avg_salary
0,1000,Person1,None,None
1,1001,Person2,Doctor,80000.00


# LEFT OUTER JOIN (models reversed)
i.e right join

In [17]:
q = """
Select * FROM occupation o
LEFT OUTER JOIN person p
on p.occupation_title == o.title
"""
sqlq =  sql_raw(q) 
je = sql_to_df(sqlq)
print(ji.shape)
je

(1, 5)


,title,avg_salary,person_id,name,occupation_title
0,School Teacher,40000,NaN,None,None
1,Doctor,80000,1001.0,Person2,Doctor


In [18]:
ormq =  Occupation.objects.values("title", "avg_salary", person_id=F("person__person_id"),
        name=F("person__name"), occupation_title=F("person__occupation_title"),)
print(ormq.query)

SELECT "occupation"."title", "occupation"."avg_salary", "person"."person_id" AS "person_id", "person"."name" AS "name", "person"."occupation_title" AS "occupation_title" FROM "occupation" LEFT OUTER JOIN "person" ON ("occupation"."title" = "person"."occupation_title")


In [19]:
print(orm_to_df(ormq).shape)
orm_to_df(ormq)

(2, 5)


,title,avg_salary,person_id,name,occupation_title
0,School Teacher,40000.00,NaN,None,None
1,Doctor,80000.00,1001.0,Person2,Doctor


# RIGHT OUTER JOIN

In [38]:
q = """
Select * FROM person p
RIGHT OUTER JOIN occupation o
on o.title == p.occupation_title
"""
sqlq =  sql_raw(q) 
jr = sql_to_df(sqlq)
print(ji.shape)
ji

(2, 5)


,person_id,name,occupation_title,title,avg_salary
0,1001.0,Person2,Doctor,Doctor,80000
1,NaN,None,None,School Teacher,40000


In [28]:
ormq =  Occupation.objects.order_by(F('person__person_id').asc(nulls_last=True))\
.values("title", "avg_salary", person_id=F("person__person_id"),
        name=F("person__name"), occupation_title=F("person__occupation_title"))
print(ormq.query)

SELECT "occupation"."title", "occupation"."avg_salary", "person"."person_id" AS "person_id", "person"."name" AS "name", "person"."occupation_title" AS "occupation_title" FROM "occupation" LEFT OUTER JOIN "person" ON ("occupation"."title" = "person"."occupation_title") ORDER BY "person"."person_id" ASC NULLS LAST


In [29]:
print(orm_to_df(ormq).shape)
orm_to_df(ormq)

(2, 5)


,title,avg_salary,person_id,name,occupation_title
0,Doctor,80000.00,1001.0,Person2,Doctor
1,School Teacher,40000.00,NaN,None,None


# FULL OUTER JOIN
Could't find any orm only solution